**Referência:** https://www.youtube.com/watch?v=ykHxllpCcQw&lc=UgzoNBuPMawwbrBbzvV4AaABAg.9LVoA0shiWs9LWVDUCNaGv

In [13]:
tickers = ['TAEE4.SA', 'ITSA4.SA']
periodo = '1y'

# Baixando Cotações e Calculando Retornos

In [2]:
!pip install -q yfinance

     |████████████████████████████████| 5.5MB 12.4MB/s 


In [14]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go

from ipywidgets import interact

In [15]:
# consertar nome das colunas
def fix_col_names(df):
  return ['IBOV' if col == '^BVSP' else col.rstrip('.SA') for col in df.columns]

In [19]:
prices = yf.download(tickers, period=periodo, rounding=True)['Adj Close']
# aplica a funcao para consertar o nome das colunas da variável preco
prices.columns = fix_col_names(prices)
# dropa os valores faltantes
prices.dropna(inplace=True)
retorno = prices.pct_change().dropna()
ativos = retorno.columns.to_list()

[*********************100%***********************]  2 of 2 completed


In [20]:
# precos mais recentes
prices.tail()

,ITSA4,TAEE4
Date,,
2021-03-24,10.03,11.90
2021-03-25,10.24,11.98
2021-03-26,10.36,12.06
2021-03-29,10.32,12.80
2021-03-30,10.41,12.95


In [21]:
# retornos mais recentes
retorno.tail()

,ITSA4,TAEE4
Date,,
2021-03-24,-0.020508,-0.019769
2021-03-25,0.020937,0.006723
2021-03-26,0.011719,0.006678
2021-03-29,-0.003861,0.061360
2021-03-30,0.008721,0.011719


# 1. Criando a carteira

In [25]:
# funcao para pegar os pesos
def calcula_carteira(df, w1):
  # peso de cada carteira. lembrando q w1 + w2 + w3 <= 1
  pesos = [w1, 1-w1]
  # cria um df com a copia das transpostas dos pesos com df
  df2 = df.dot(pesos).copy()
  # df2.mean() * 252 anualiza, df2.std() * np.sqrt(252) tb anualiza
  return df2.mean() * 252, df2.std() * np.sqrt(252)

In [26]:
# CRiando 100 carteiras de tres ativos com pesos variaveis
carteira = pd.DataFrame()
for i in np.linspace(0, 1, 101):
  media, std = calcula_carteira(retorno, i)
  carteira.at[i, 'retorno'] = media
  carteira.at[i, 'volatilidade'] = std

carteira

,retorno,volatilidade
0.00,0.584224,0.235519
0.01,0.581036,0.234393
0.02,0.577849,0.233319
0.03,0.574662,0.232299
0.04,0.571475,0.231333
...,...,...
0.96,0.278244,0.358318
0.97,0.275056,0.361202
0.98,0.271869,0.364101
0.99,0.268682,0.367015


In [27]:
# Carteira de volatilidade mínima
min_vol_idx = carteira['volatilidade'].idxmin()

In [28]:
# Calcula a fronteira eficiente
if (carteira['retorno'].iloc[0] > carteira['retorno'].iloc[-1]):
  fe = carteira.loc[:min_vol_idx, :]
else:
  fe = carteira.loc[min_vol_idx:, :]

# 2. Gráfico

In [40]:
def gerar_grafico(w1, mostrar_curvas):
  fig = go.Figure()

  # Desenha um ponto com o retorno e a volatilidade da carteira
  fig.add_scatter(y=[carteira.iloc[w1]['retorno']],
                  x=[carteira.iloc[w1]['volatilidade']],
                  text=['CARTEIRA'],
                  mode='markers+text', name='CARTEIRA')
  
  # Desenha os pontos das ações individuais
  fig.add_scatter(y=carteira.iloc[[-1, 0]]['retorno'],
                  x=carteira.iloc[[-1, 0]]['volatilidade'],
                  text=ativos,
                  mode='markers+text', name='Ações')
  
  # Desenha a curva
  fig.add_scatter(y=carteira['retorno'],
                   x=carteira['volatilidade'],
                   mode='lines', name='Curva',
                   visible=mostrar_curvas)
  
  # Plotar a carteira de volatilidade mínima
  fig.add_scatter(y=[carteira.loc[min_vol_idx]['retorno']],
                   x=[carteira.loc[min_vol_idx]['volatilidade']],
                   mode='markers',
                   name='Carteira de Mínima Variância', 
                   visible=mostrar_curvas)
   
   # Desenha a fronteira eficiente
  fig.add_scatter(y=fe['retorno'],
                   x=fe['volatilidade'],
                   mode='lines', name='Fronteira Eficiente', 
                   line={'color': 'red'},
                   visible=mostrar_curvas)
   
  fig.update_traces(textfont_size=12,
                     textposition='middle right', 
                     textfont_color='white',
                     hovertemplate='<b>retorno: </b> %{y:.1%}' + 
                                    '<br><b>volatilidade:</b> %{x:.1%}')
   
  fig.layout.autosize = False
  fig.layout.xaxis.title = 'Volatilidade'
  fig.layout.yaxis.title = 'Retorno Esperado'
  fig.layout.yaxis.range = [carteira['volatilidade'].min()-0.05, carteira['volatilidade'].max()+0.05]
  fig.layout.yaxis.range = [carteira['retorno'].min()-0.05, carteira['retorno'].max()+0.05]
  fig.layout.xaxis.tickformat = '.0%'
  fig.layout.yaxis.tickformat = '.0%'
  fig.layout.title = f"<b>{ativos[0]}:</b> {w1}% <b>{ativos[1]}:</b> {100-w1}%"
  fig.layout.template = 'plotly_dark'

  fig.show(config=dict(
       displayModeBar=True
   ))

# Dispara o grafico  
interact(gerar_grafico, w1=(0,100,1), mostrar_curvas=False);

interactive(children=(IntSlider(value=50, description='w1'), Checkbox(value=False, description='mostrar_curvas…